In [33]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

In [34]:
df= pd.read_csv("ASTD.csv")
df2= pd.read_csv("ASTD.csv")
df.head()

,sentiment,txt
0,0,عن اي مشروب غازي كوﻻ #حافظ_مش_فاهم
1,0,عادة لا اعلن عن مواقفي السياسيةلكن كله الا الد...
2,1,إنتا فرحة جات لِ عندي بعد دنيا من التعب السعاد...
3,1,الببلاوي: تصنيف الإخوان تنظيما إرهابيا قرار سي...
4,1,1 فكرة المقالة حلوة وأسلوبك في توصيل الفكرة حل...


In [73]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 2 columns):
sentiment    1589 non-null int64
txt          1589 non-null object
dtypes: int64(1), object(1)
memory usage: 24.9+ KB


In [36]:
def cleaner(text):
    text = text.lower()
    text = re.sub("@[^\s]+","",text)
    text = text.replace(":)","")  
    text = text.replace("@","") 
    text = text.replace("#","") 
    text = text.replace(":(","")
    return text

In [37]:
def remove_stop_words(text):
    sw = stopwords.words("arabic")
    clean_words = []
    text = text.split()
    for word in text:
        if word not in sw:
            clean_words.append(word)
    return " ".join(clean_words)

In [38]:
def stemming(text):
    ps = PorterStemmer()
    text = text.split()
    stemmed_words = []
    for word in text :
        stemmed_words.append(ps.stem(word))
    return " ".join(stemmed_words)

In [39]:
def run(text):
    text = cleaner(text)
    text = remove_stop_words(text)
    text = stemming(text)
    return text

In [40]:
df['txt'] = df['txt'].apply(run)

In [60]:
df2.iloc[1,1]

'عادة لا اعلن عن مواقفي السياسيةلكن كله الا الدستور في كلمة واحدة:مهزلةو لو وصلنا لمهزلة الاستفتاء حقول لا'

In [42]:
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(df["txt"]).toarray()


In [51]:
y = df['sentiment'].values
y

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [74]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33)

In [75]:
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
y_pred = lr.predict(x_test)

In [77]:
lr.score(x_test,y_test)

0.7695238095238095

In [78]:
test = "عادة لا اعلن عن مواقفي السياسيةلكن كله الا الدستور في كلمة واحدة:مهزلةو لو وصلنا لمهزلة الاستفتاء حقول لا"
test = run(test)
test = tfidf.transform([test]).toarray()

In [79]:
lr.predict(test)

array([0], dtype=int64)

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

             precision    recall  f1-score   support

          0       0.80      0.77      0.78       284
          1       0.74      0.77      0.75       241

avg / total       0.77      0.77      0.77       525



In [81]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)


array([[218,  66],
       [ 55, 186]], dtype=int64)